In [1]:
import os
if not os.path.exists('BIGG'):
    os.makedirs('BIGG')

# Modify origin json file

In [1]:
# modified from http://bigg.ucsd.edu/static/namespace/universal_model.json to a cobrapy json file

def add_to_dir(my_dir, key, value):
    if key in my_dir:
        if isinstance(my_dir[key], list):
            my_dir[key].append(value)
        else:
            my_dir[key] = [my_dir[key], value]
    else:
        my_dir[key] = value


import json

with open('BIGG\\universal_model.json', 'r') as f:
    data = json.load(f)

for item in data['metabolites']+data['reactions']:
    annolist = item.get('annotation')
    if annolist:
        annodir={}
        for anno in annolist:
            spli = anno[1].split('/')
            key = spli[-2]
            value = spli[-1]
            add_to_dir(annodir, key, value)
        item['annotation'] = annodir
    else:
        item['annotation'] = {}

with open('BIGG\\universal_model_modified.json', 'w') as f:
    json.dump(data, f)

# Get relation betweent other database and BIGG

In [1]:
import cobra
biggunimodel = cobra.io.load_json_model('BIGG\\universal_model_modified.json')
#cobra.io.write_sbml_model(biggunimodel,"BIGG\\universal_model_modified.xml")
biggunimodel

Name,bigg_universal
Memory address,2806eac56a0
Number of metabolites,15638
Number of reactions,28301
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [2]:
# get the list of other database reactions with a certain annotation

def add_to_dir(my_dir, key, value):
    if key in my_dir:
        if isinstance(my_dir[key], list):
            my_dir[key].append(value)
        else:
            my_dir[key] = [my_dir[key], value]
    else:
        my_dir[key] = value

        
KEGG_BIGG_dir = {}
for rxn in biggunimodel.reactions:
    kegganno = rxn.annotation.get('kegg.reaction')
    if kegganno:
        if type(kegganno) is list:
            for i in kegganno:
                add_to_dir(KEGG_BIGG_dir, i, rxn.id)
        else:
            add_to_dir(KEGG_BIGG_dir, kegganno, rxn.id)

EC_BIGG_dir = {}
for rxn in biggunimodel.reactions:
    ecanno = rxn.annotation.get('ec-code')
    if ecanno:
        if type(ecanno) is list:
            for i in ecanno:
                add_to_dir(EC_BIGG_dir, i, rxn.id)
        else:
            add_to_dir(EC_BIGG_dir, ecanno, rxn.id)

cyc_BIGG_dir = {}
for rxn in biggunimodel.reactions:
    cycanno = rxn.annotation.get('biocyc')
    if cycanno:
        if type(cycanno) is list:
            for i in cycanno:
                add_to_dir(cyc_BIGG_dir, i, rxn.id)
        else:
            add_to_dir(cyc_BIGG_dir, cycanno, rxn.id)

seed_BIGG_dir = {}
for rxn in biggunimodel.reactions:
    seedanno = rxn.annotation.get('seed.reaction')
    if seedanno:
        if type(seedanno) is list:
            for i in seedanno:
                add_to_dir(seed_BIGG_dir, i, rxn.id)
        else:
            add_to_dir(seed_BIGG_dir, seedanno, rxn.id)

In [53]:
def select_rxn(biggunimodel, rxnlist):
    if not isinstance(rxnlist,list):
        unirxn = biggunimodel.reactions.get_by_id(rxnlist)
        comlist=[]
        for key, value in unirxn.metabolites.items():
            comlist.append(key.id.split('_')[-1])
        if set(comlist).issubset(set(['c','e','p'])):
            return rxnlist
        else:
            return None
    else:
        newrxnlist_c = []
        newrxnlist_p = []
        newrxnlist_e = []
        for rxn in rxnlist:
            unirxn = biggunimodel.reactions.get_by_id(rxn)
            comlist=[]
            for key, value in unirxn.metabolites.items():
                comlist.append(key.id.split('_')[-1])
            if set(comlist).issubset(set(['c'])):
                newrxnlist_c.append(rxn)
            elif set(comlist).issubset(set(['c','p'])):
                newrxnlist_p.append(rxn)
            elif set(comlist).issubset(set(['c','e','p'])):
                newrxnlist_e.append(rxn)
        if len(newrxnlist_c) > 0:
            return min(newrxnlist_c, key=len)
        elif len(newrxnlist_p) > 0:
            return min(newrxnlist_p, key=len)
        elif len(newrxnlist_p) > 0:
                return min(newrxnlist_e, key=len)
        else:
            return None
        
def kegg2bigg(keggid,biggunimodel=biggunimodel,KEGG_BIGG_dir=KEGG_BIGG_dir):
    if KEGG_BIGG_dir.get(keggid):
        selr = select_rxn(biggunimodel, KEGG_BIGG_dir.get(keggid))
        if selr:
            if type(selr) is list:
                print(keggid)
                for i in selr:
                    print('     http://bigg.ucsd.edu/universal/reactions/'+i)
                    #print('KEGG_BIGG_dir_selected['+'\''+keggid+'\''+'] = '+'\''+i+'\'')
                #print(keggid, selr)
                return selr
            elif type(selr) is str:
                return selr     
    else:
        return None        

def bigg2kegg(biggid,biggunimodel=biggunimodel):
    try:
        rxn = biggunimodel.reactions.get_by_id(biggid)
    except:
        return ''
    if rxn.annotation.get('kegg.reaction'):
        return rxn.annotation.get('kegg.reaction')
    else:
        return ''  
    
def seed2bigg(seedid,biggunimodel=biggunimodel,seed_BIGG_dir=seed_BIGG_dir):
    if seed_BIGG_dir.get(seedid):
        selr = select_rxn(biggunimodel, seed_BIGG_dir.get(seedid))
        if selr:
            if type(selr) is list:
                print(seedid, selr)
                return selr
            elif type(selr) is str:
                return selr     
    else:
        return None

In [55]:
KEGG_BIGG_dir_selected = {}
for key, value in KEGG_BIGG_dir.items():
    new_value = kegg2bigg(key)
    if new_value:
        KEGG_BIGG_dir_selected[key] = new_value

In [56]:
import json
with open('KEGG\\KEGG_BIGG_dir_selected.json', 'w') as f:
    f.write(json.dumps(KEGG_BIGG_dir_selected))
with open('BIGG\\KEGG_BIGG_dir_selected.json', 'w') as f:
    f.write(json.dumps(KEGG_BIGG_dir_selected))

In [12]:
seed_BIGG_dir_selected = {}
for key, value in seed_BIGG_dir.items():
    new_value = seed2bigg(key)
    if new_value:
        seed_BIGG_dir_selected[key] = new_value

rxn09806 ['2DHPtm', '2DHPtn']
rxn09802 ['34HPPt2p', '34HPPt2m']
rxn09869 ['34HPPt2p', '34HPPt2m']
rxn09793 ['4H2OGLTtp', '4H2OGLTtm']
rxn09868 ['4H2OGLTtp', '4H2OGLTtm']
rxn09648 ['6PGLter', '6PGLthr']
rxn13213 ['ACCOAtn', 'ACCOAgt', 'ACCOAtr', 'ACCOAts', 'ACCOAtm_1']
rxn09865 ['ASPGLU2m', 'ASPGLUtp']
rxn13231 ['ASPGLU2m', 'ASPGLUtp']
rxn09528 ['ATPS3m', 'ATPS3g', 'ATPS3v']
rxn09529 ['ATPS3m', 'ATPS3g', 'ATPS3v']
rxn09530 ['ATPS3m', 'ATPS3g', 'ATPS3v']
rxn13766 ['ATPS3m', 'ATPS3g', 'ATPS3v']
rxn09780 ['ATPtm_H', 'ATPtp_H']
rxn09863 ['ATPtm_H', 'ATPtp_H']
rxn09779 ['CITtap', 'CITtam']
rxn09862 ['CITtap', 'CITtam']
rxn09777 ['CITtcm', 'CITtcp']
rxn09861 ['CITtcm', 'CITtcp']
rxn13290 ['FRDPtm', 'FRDPtc', 'FRDPth', 'FRDPtcr', 'FRDPtr']
rxn13332 ['IPDPtm', 'IPDPtr', 'IPDPtx', 'IPDPth']
rxn13407 ['TRDRDtp', 'r1441']
rxn09483 ['ASPOcm_1', 'ASPOcm']
rxn09804 ['2OMPH_5tn', '2OMPH_5tm']
rxn09829 ['2OMPH_5tn', '2OMPH_5tm']
rxn09817 ['DHORtn', 'DHORStm']
rxn12647 ['DM_datp_m', 'DATPtn', 'DM_datp_n

In [13]:
del_list = ['rxn09806','rxn09802','rxn09869','rxn09793','rxn09868','rxn09648','rxn13213','rxn09865',
            'rxn13231','rxn09528','rxn09529','rxn09530','rxn13766','rxn09780','rxn09863','rxn09779',
            'rxn09862','rxn09777','rxn09861','rxn13290','rxn13332','rxn13407','rxn09483','rxn09804',
            'rxn09829','rxn09817','rxn12647','rxn09836','rxn00011','rxn00544','rxn02271','rxn09774',
            'rxn09859','rxn13314','rxn02342','rxn00738','rxn01523','rxn02867','rxn06586','rxn03430']
for key in del_list:
    del seed_BIGG_dir_selected[key]


In [14]:
with open('BIGG\\SEED_BIGG_dir_selected.json', 'w') as f:
    f.write(json.dumps(seed_BIGG_dir_selected))

# Draw BIGG label to KEGG pathway map pictures

In [1]:
import myModules.KEGG2BIGG as K2B
unikegglist = list(set(K2B.KEGG_pathways_reactions['R_ID'].tolist()))
mappedinBIGG = [r for r in unikegglist if K2B.KEGG_BIGG_dir_selected.get(r)]
a = K2B.pd.read_csv('KEGG\\Reaction_in_pathway_count.csv',index_col=0,squeeze=True)

C:\Users\lix\AppData\Local\Temp\ipykernel_17980\3013797931.py:4: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  a = K2B.pd.read_csv('KEGG\\Reaction_in_pathway_count.csv',index_col=0,squeeze=True)


In [2]:
b = K2B.pathway_count(mappedinBIGG,a)

map00230 104 / 153 Purine metabolism
map00240 73 / 99 Pyrimidine metabolism
map00270 60 / 102 Cysteine and methionine metabolism
map00520 49 / 146 Amino sugar and nucleotide sugar metabolism
map00860 49 / 171 Porphyrin metabolism
map00330 48 / 95 Arginine and proline metabolism
map00630 47 / 83 Glyoxylate and dicarboxylate metabolism
map00620 45 / 82 Pyruvate metabolism
map00260 44 / 72 Glycine, serine and threonine metabolism
map00680 39 / 105 Methane metabolism
map00040 34 / 77 Pentose and glucuronate interconversions
map00250 33 / 46 Alanine, aspartate and glutamate metabolism
map00760 33 / 75 Nicotinate and nicotinamide metabolism
map00720 32 / 57 Carbon fixation pathways in prokaryotes
map00790 31 / 69 Folate biosynthesis
map00350 31 / 100 Tyrosine metabolism
map00650 31 / 69 Butanoate metabolism
map00640 30 / 64 Propanoate metabolism
map00071 29 / 46 Fatty acid degradation
map00400 28 / 48 Phenylalanine, tyrosine and tryptophan biosynthesis
map00670 27 / 31 One carbon pool by fol

In [3]:
from PIL import Image, ImageFont, ImageDraw

def drawtext(draw,text,coords_list):
    box = draw.textbbox((0,0),text, font=font)
    w = box[2] - box[0]
    h = box[3] - box[1]
    if type(coords_list) == list:
        coords_num = [2*int(i) for i in coords_list]
        #draw.rectangle(coords_num, fill=(255, 192, 203), outline=(5, 5, 5),width=1)
        #draw.text(((coords_num[0]+coords_num[2])/2-w/2,(coords_num[3]+coords_num[1])/2-h/2), text, font=font,fill=(0, 0, 0))
        draw.text(((coords_num[0]+coords_num[2])/2-w/2,coords_num[3]), text, font=font,fill=(218, 155, 0))
    else:
        print('coords_list should be a list')


map_list=[key for key in b.keys()]
#map_list = ['map00680']
font = ImageFont.truetype("arial.ttf", 15)
for mapp in map_list:
    map_reactions = K2B.KEGG_pathways_reactions[K2B.KEGG_pathways_reactions['pw_ID']==mapp]
    inBIGG_map_reactions = [r for r in mappedinBIGG if r in map_reactions['R_ID'].to_list()]
    img = Image.open("KEGG\\maps\\mapasR\\%s_asR.png"%mapp)
    draw = ImageDraw.Draw(img)
    for r in inBIGG_map_reactions:
        text = K2B.KEGG_BIGG_dir_selected.get(r)
        findresult = map_reactions[map_reactions['R_ID']==r]['coords'].to_list()
        for i in findresult:
            coords_list = i.split(',')
            if len(coords_list) == 4:
                drawtext(draw,text,coords_list) 
    img.save('KEGG\\maps\\mapedinBIGGandR\\%s_biggandR.png'%mapp)
